<a href="https://colab.research.google.com/github/paulovbpo/machine_learning_alura/blob/master/recomendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carrega os Dados

In [4]:
import pandas as pd
filmes = pd.read_csv('movies.csv')
filmes.columns = ['Id_Filme','Título','Gêneros'] 
filmes = filmes.set_index('Id_Filme')  # método para mudar o índice de uma coluna
filmes.head()

,Título,Gêneros
Id_Filme,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [5]:
notas = pd.read_csv('ratings.csv')
notas.columns = ['Id_Usuário','Id_Filme','Nota','Momento']
notas.describe()   # método que retorna importantes informações numérica dos dados

,Id_Usuário,Id_Filme,Nota,Momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


# 1° Tentativa de Recomendação:
   **Baseado no Número de Votos**

In [6]:
total_votos = notas['Id_Filme'].value_counts()   # conta o numero de vezes que foi votado cada filme
filmes['Total_Votos'] = total_votos     # atribui a series(variável) criada anteriormente a uma nova coluna do DF filmes
filmes.sort_values('Total_Votos', ascending = False).head()  # ordena de forma decrescente o total de votos

,Título,Gêneros,Total_Votos
Id_Filme,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0


In [7]:
filmes.loc[700]   # método para localizar determinado dado

Título         Angus (1995)
Gêneros              Comedy
Total_Votos               7
Name: 700, dtype: object

# 2° Tentativa de Recomendação:
  **Baseado na Média das Notas**

In [8]:
media_notas = notas.groupby('Id_Filme').mean()['Nota']  # agrupa a coluna do Id_Filme pela media da coluna 'Nota'
filmes['Média_Notas'] = media_notas  # atribui a series(variável) criada anteriormente a uma nova coluna do DF filmes
filmes.sort_values('Média_Notas', ascending = False).head() # ordena de forma decrescente a média de notas 

,Título,Gêneros,Total_Votos,Média_Notas
Id_Filme,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0


# 3° Tentativa de Recomendação:
  **Média das Notas com Filtragem dos Números de Votos**

In [9]:
filmes_mais_50_votos = filmes.query('Total_Votos >= 50')
filmes_mais_50_votos.sort_values('Média_Notas', ascending = False).head() 
# método query filtra o conjunto de dados baseado em uma condição 

,Título,Gêneros,Total_Votos,Média_Notas
Id_Filme,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041


# 4° Tentativa de Recomendação
**Filmes assistidos e Gêneros**

In [10]:
filmes_assistidos = [1, 17, 19, 22, 29, 44, 66]
filmes.loc[filmes_assistidos]

,Título,Gêneros,Total_Votos,Média_Notas
Id_Filme,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
17,Sense and Sensibility (1995),Drama|Romance,67.0,3.776119
19,Ace Ventura: When Nature Calls (1995),Comedy,88.0,2.727273
22,Copycat (1995),Crime|Drama|Horror|Mystery|Thriller,36.0,3.222222
29,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,38.0,4.013158
44,Mortal Kombat (1995),Action|Adventure|Fantasy,46.0,2.543478
66,Lawnmower Man 2: Beyond Cyberspace (1996),Action|Sci-Fi|Thriller,9.0,2.500000


In [11]:
# filtra os generos iguais a estes e filmes com mais de 50 votos
acao_aventura_fantasia = filmes_mais_50_votos.query("Gêneros == 'Action|Adventure|Fantasy'")

acao_aventura_fantasia.drop(filmes_assistidos, errors = 'ignore').sort_values('Média_Notas', ascending = False).head()
# método drop para retirar filmes desses gêneros que ja foram assitidos, e ignorar os erros pois dentro dos filmes assistidos existem de outros generos

,Título,Gêneros,Total_Votos,Média_Notas
Id_Filme,,,,
2115,Indiana Jones and the Temple of Doom (1984),Action|Adventure|Fantasy,108.0,3.638889
45722,Pirates of the Caribbean: Dead Man's Chest (2006),Action|Adventure|Fantasy,72.0,3.506944
653,Dragonheart (1996),Action|Adventure|Fantasy,65.0,3.092308


# 5° Tentativa de Recomendação
**Filmes similares que outras pessoas assistiram**

In [0]:
import numpy as np
# função para calcular a distância entre dois vetores
def distancia_vetores(a,b):
  return np.linalg.norm(a - b)

In [0]:
# função para filtrar as notas de um usuário específico
def notas_usuario(usuario):
  notas_usuario = notas.query("Id_Usuário == %d" %usuario) 
  notas_usuario = notas_usuario[['Id_Filme','Nota']].set_index('Id_Filme')  # seleciona apenas as colunas do IdFilme e Nota, além de substituir o index
  return notas_usuario

In [0]:
# função para calcular a distância entre dois usuários
def distancia_usuarios(user_1,user_2):
  notas1 = notas_usuario(user_1)
  notas2 = notas_usuario(user_2)
  diferencas = notas1.join(notas2, lsuffix = "_Esquerda", rsuffix="_Direita").dropna() # método dropna para exlcuir os filmes que apenas o usuario1 assistiu
  # método join para juntar os DF, tendo que passar sufixos para o nome da coluna Nota para diferencia-las
  
  
  minimo = 5
  if(len(diferencas)< minimo):    # para desconsiderar usuarios que possuam um mínimo de filmes em comum
   return [user_1, user_2, 9999] 
  #  return None
    
  
  distancia = distancia_vetores(diferencas['Nota_Esquerda'],diferencas['Nota_Direita'])
  return [user_1, user_2, distancia]

In [0]:
# função para calcular a distância entre um usuário e o resto
def distancia_todos(usuario):
  todos_usuarios = notas['Id_Usuário'].unique()   # método unique para receber todos os usuários do DF
  distancias = [distancia_usuarios(usuario, usuario_id) for usuario_id in todos_usuarios] # um loop para percorrer cada usuario
  distancias = pd.DataFrame(distancias, columns = ['Usuário_Escolhido','Todos_Usuários','Distância'])  # transforma a variável distancias em uma DF para poder maniupala-la melhor  
  return distancias

In [0]:
# função para ordenar a distancia entre os usuários
def ordem_distancia(usuario):
  distancias = distancia_todos(usuario)
  distancias = distancias.sort_values('Distância')  # coloca em ordem crescente
  distancias = distancias.set_index('Todos_Usuários').drop(usuario)  # coloca a coluna de todos os usuarios como índice e retira a 1° linha para não comparar com o proprio usuario
  return distancias

In [17]:
distancia_todos(1).head()

,Usuário_Escolhido,Todos_Usuários,Distância
0,1,1,0.000000
1,1,2,9999.000000
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657


# 6° Tentativa de Recomendação
**Filmes similares da pessoa mais próxima**

In [0]:
# função para achar usuários com notas mais próximas, passando a quantidade de usuários que é para ser analisado
def mais_proximos(usuario, num_usuarios = None):
  distancias = distancia_todos(usuario, num_usuarios = num_usuarios)
  distancias = distancias.sort_values('Distância')  # coloca em ordem crescente
  distancias = distancias.set_index('Todos_Usuários').drop(usuario)  # coloca a coluna de todos os usuarios como índice e retira a 1° linha para não comparar com o proprio usuario
  return distancias

In [0]:
def distancia_todos(usuario, num_usuarios = None):
  todos_usuarios = notas['Id_Usuário'].unique()   # método unique para receber todos os usuários do DF
  if num_usuarios:
    todos_usuarios = todos_usuarios[:num_usuarios]   # para percorrer até o número desejado de usuários
    
  distancias = [distancia_usuarios(usuario, usuario_id) for usuario_id in todos_usuarios] # um loop para percorrer cada usuario
  distancias = list(filter(None,distancias))    # para filtrar os usuários que possuem muito pouco em comum
  distancias = pd.DataFrame(distancias, columns = ['Usuário_Escolhido','Todos_Usuários','Distância'])  # transforma a variável distancias em uma DF para poder maniupala-la melhor  
  return distancias

In [0]:
# função que recomenda baseado no que a pessoa mais similar a ela viu
def recomendar(usuario, num_usuarios = None):
  notas_usuario_escolhido = notas_usuario(usuario)  # pega as notas do usuario escolhido
  filmes_vistos = notas_usuario_escolhido.index     # extrai os filmes ja vistos por ele


  similares = mais_proximos(usuario, num_usuarios = num_usuarios) # pega os usuarios mais similares
  similar = similares.iloc[0].name  # pega o índice (name) da primeira linha(iloc), para selecionar o usuário mais similar
  notas_similar = notas_usuario(similar)  # extrai as notas do usuario mais similiar
  notas_similar = notas_similar.drop(filmes_vistos, errors = 'ignore')  # retira os filmes ja vistos pelo usuario escolhido
  recomendacao = notas_similar.sort_values('Nota', ascending = False).join(filmes)  # ordena de acordo com as notas que a pessoa similar deu, e junta com o DF de filmes(1° criado)
  return recomendacao

In [21]:
recomendar(1).head()

,Nota,Título,Gêneros,Total_Votos,Média_Notas
Id_Filme,,,,,
8636,5.0,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,79.0,3.803797
58559,5.0,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149.0,4.238255
33794,5.0,Batman Begins (2005),Action|Crime|IMAX,116.0,3.862069
4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,198.0,4.106061
5349,5.0,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller,122.0,3.540984


# 7° Tentativa de Recomendação
**Filmes similares DAS PESSOAS mais próximas**

In [0]:
# praticamente igual a função mais_proximos porem com o nome mais usual(também conhecido como KNN), além de passar a quantidade de usuários mais próximos
def k_nearest_neighbors(usuario, k_mais_proximos = 10 ,num_usuarios = None):
  distancias = distancia_todos(usuario, num_usuarios = num_usuarios)
  distancias = distancias.sort_values('Distância')  # coloca em ordem crescente
  distancias = distancias.set_index('Todos_Usuários').drop(usuario)  # coloca a coluna de todos os usuarios como índice e retira a 1° linha para não comparar com o proprio usuario
  return distancias.head(k_mais_proximos)

In [0]:
# função que recomenda baseado no que as pessoas mais similares a ela viram
def recomendam(usuario, k_mais_proximos = 10, num_usuarios = None):
  notas_usuario_escolhido = notas_usuario(usuario)  # pega as notas do usuario escolhido
  filmes_vistos = notas_usuario_escolhido.index     # extrai os filmes ja vistos por ele

  similares = k_nearest_neighbors(usuario, k_mais_proximos = k_mais_proximos , num_usuarios = num_usuarios) # pega os usuarios mais similares
  usuarios_similares = similares.index   # pega o índice dos usuários mais similares
  notas_similares = notas.set_index('Id_Usuário').loc[usuarios_similares]  # dentro do Id_Usuario, do DF notas, localizar os usuários similares 
  recomendacao = notas_similares.groupby('Id_Filme').mean()[['Nota']]  # agrupa pelos filmes e retorna a média das notas dos usuários similares, usando dois colchetes para retornar um DF
  recomendacao = recomendacao.sort_values('Nota', ascending = False).join(filmes) 
  return recomendacao

In [24]:
recomendam(1).head()

,Nota,Título,Gêneros,Total_Votos,Média_Notas
Id_Filme,,,,,
1704,5.0,Good Will Hunting (1997),Drama|Romance,141.0,4.078014
57504,5.0,"Girl Who Leapt Through Time, The (Toki o kaker...",Animation|Comedy|Drama|Romance|Sci-Fi,10.0,4.100000
38304,5.0,No Direction Home: Bob Dylan (2005),Documentary,4.0,4.625000
48394,5.0,"Pan's Labyrinth (Laberinto del fauno, El) (2006)",Drama|Fantasy|Thriller,81.0,3.814815
2300,5.0,"Producers, The (1968)",Comedy,33.0,3.969697


# Testar um Novo Usuário

In [0]:
# função para adicionar um novo usuário (com filmes e notas dadas) no DataFrame notas
def novo_usuario(dados):
  novo_usuario = notas['Id_Usuário'].max()+1   # cria o Id do novo usuário, que será o último Id existente + 1
  notas_novo_usuario = pd.DataFrame(dados,columns = ['Id_Filme','Nota'])  # cria um DF com os filmes e notas do novo usuário  
  notas_novo_usuario['Id_Usuário'] = novo_usuario   # adiciona o Id criado as notas e filmes dele
  return pd.concat([notas,notas_novo_usuario], sort=True)   # junta o DF das notas do usuario novo ao DF de notas, entre colchetes pois recebe uma lista

In [26]:
notas = novo_usuario([[58559,2],[33794,2],[5349,2],[8636,2],[1704,5]])
notas.tail(10)

,Id_Filme,Id_Usuário,Momento,Nota
100831,166534,610,1.493848e+09,4.0
100832,168248,610,1.493850e+09,5.0
100833,168250,610,1.494273e+09,5.0
100834,168252,610,1.493846e+09,5.0
100835,170875,610,1.493846e+09,3.0
0,58559,611,NaN,2.0
1,33794,611,NaN,2.0
2,5349,611,NaN,2.0
3,8636,611,NaN,2.0
4,1704,611,NaN,5.0


In [27]:
recomendam(612).head()

KeyError: ignored

# Filtragem
**KNN para Filmes com Mais de 50 Votos**

In [28]:
notas = notas.set_index('Id_Filme').loc[filmes_mais_50_votos.index]   # substitui o índice para ser Id_Filme
notas.head()

,Id_Usuário,Momento,Nota
Id_Filme,,,
1,1,9.649827e+08,4.0
1,5,8.474350e+08,4.0
1,7,1.106636e+09,4.5
1,15,1.510578e+09,2.5
1,17,1.305696e+09,4.5


In [29]:
notas = notas.reset_index()    # reseta o índice
notas.head()

,Id_Filme,Id_Usuário,Momento,Nota
0,1,1,9.649827e+08,4.0
1,1,5,8.474350e+08,4.0
2,1,7,1.106636e+09,4.5
3,1,15,1.510578e+09,2.5
4,1,17,1.305696e+09,4.5


**Mínimo de usuários com recomendações**

In [0]:
def k_nearest_neighbors(usuario, k_mais_proximos = 10 ,num_usuarios = None):
  distancias = distancia_todos(usuario, num_usuarios = num_usuarios)
  distancias = distancias.sort_values('Distância')  
  distancias = distancias.set_index('Todos_Usuários').drop(usuario, errors = 'ignore')    # adiciona o 'errors = 'ignore'' 
  return distancias.head(k_mais_proximos)

In [0]:
def recomendam(usuario, k_mais_proximos = 10, num_usuarios = None):
  notas_usuario_escolhido = notas_usuario(usuario) 
  filmes_vistos = notas_usuario_escolhido.index     

  similares = k_nearest_neighbors(usuario, k_mais_proximos = k_mais_proximos , num_usuarios = num_usuarios)
  usuarios_similares = similares.index
  notas_similares = notas.set_index('Id_Usuário').loc[usuarios_similares]
  recomendacao = notas_similares.groupby('Id_Filme').mean()[['Nota']]
  aparicoes = notas_similares.groupby('Id_Filme').count()[['Nota']]   # conta o número de vezes que cada filme é votado pelos usuários similares e transforma aparicoes em DF
  
  filtro_minimo = k_mais_proximos/2   # 5
  recomendacao = recomendacao.join(aparicoes, lsuffix = '_Média_Usuários', rsuffix = '_Aparições_Usuários')   # junta a recomendacao com a aparições passando sufixos para diferenciar as colunas das notas
  recomendacao = recomendacao.query("Nota_Aparições_Usuários >= %.2f" %filtro_minimo)  # Logo filtra aparições menores do que 5
  recomendacao = recomendacao.sort_values('Nota_Média_Usuários', ascending = False).join(filmes)   # atualizar Nota para Nota_Média_Usuários
  recomendacao = recomendacao.drop(filmes_vistos,errors = 'ignore')  # para retirar filmes ja vistos
  return recomendacao

In [35]:
recomendam(612).head()

,Nota_Média_Usuários,Nota_Aparições_Usuários,Título,Gêneros,Total_Votos,Média_Notas
Id_Filme,,,,,,
356,4.200000,5,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
2858,4.166667,6,American Beauty (1999),Drama|Romance,204.0,4.056373
39,4.083333,6,Clueless (1995),Comedy|Romance,104.0,3.293269
1197,4.000000,6,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,142.0,4.232394
260,3.833333,6,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076


In [36]:
recomendam(612, k_mais_proximos = 20).head()

,Nota_Média_Usuários,Nota_Aparições_Usuários,Título,Gêneros,Total_Votos,Média_Notas
Id_Filme,,,,,,
356,3.769231,13,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
593,3.590909,11,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290


In [37]:
recomendam(1).head()

,Nota_Média_Usuários,Nota_Aparições_Usuários,Título,Gêneros,Total_Votos,Média_Notas
Id_Filme,,,,,,
7153,4.700000,5,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,185.0,4.118919
58559,4.700000,5,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149.0,4.238255
4993,4.666667,6,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,198.0,4.106061
318,4.642857,7,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
109487,4.583333,6,Interstellar (2014),Sci-Fi|IMAX,73.0,3.993151
